In [ ]:
pip install python-telegram-bot --upgrade

In [5]:
pip install apiai

Note: you may need to restart the kernel to use updated packages.


In [1]:
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters

In [2]:
import apiai, json

#### Создаем Updater используя Telegram API токен

In [3]:
updater = Updater(token='1047545489:AAHFiLdY_ZGqdJG6pJqH1tMCJDn2FqI6Ul0')

/Users/ellu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: TelegramDeprecationWarning: Old Handler API is deprecated - see https://git.io/fxJuV for details
  """Entry point for launching an IPython kernel.


In [4]:
dispatcher = updater.dispatcher

#### Приветствие бота

In [5]:
def startCommand(bot, update):
    bot.send_message(chat_id=update.message.chat_id, text='Привет!')

#### Для обработки сообщений нужно плучить токен Dilaogfow API

In [6]:
def textMessage(bot, update):
    request = apiai.ApiAI('453dc54ef924420bbcef24466376d7fa').text_request() # Токен API к Dialogflow
    request.lang = 'ru' # На каком языке будет послан запрос
    request.session_id = 'BatlabAIBot' # ID Сессии диалога (нужно, чтобы потом учить бота)
    request.query = update.message.text # Посылаем запрос к ИИ с сообщением от юзера
    responseJson = json.loads(request.getresponse().read().decode('utf-8')) #грузим в json 
    response = responseJson['result']['fulfillment']['speech'] # Разбираем JSON и вытаскиваем ответ
    # Если есть ответ от бота - присылаем юзеру, если нет - бот его не понял
    if response:
        bot.send_message(chat_id=update.message.chat_id, text=response)
    else:
        bot.send_message(chat_id=update.message.chat_id, text='Я Вас не совсем понял!')

## Запускаем обработчик сообщений

In [7]:
# Создаем хендлеры
start_command_handler = CommandHandler('start', startCommand)
text_message_handler = MessageHandler(Filters.text, textMessage)
# Добавляем хендлеры в диспетчер
dispatcher.add_handler(start_command_handler)
dispatcher.add_handler(text_message_handler)
# Начинаем опрос сообщений
updater.start_polling(clean=True)
# Останавливаем бота по нажатию Ctrl+C
updater.idle()